In [ ]:
!python df_concat.py

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas.plotting import scatter_matrix

from sklearn.preprocessing import StandardScaler, OneHotEncoder


from sklearn.neural_network import MLPRegressor, MLPClassifier


from sklearn.model_selection import train_test_split
import re


In [2]:


ak_model_total = pd.read_csv('clean_ak_df', dtype = {'defect_code': 'object'}, float_precision='round_trip',
                            index_col = 'index')

In [72]:
ak_model_total.head()

,price,carbon,manganese,phosphorus,sulfur,silicon,aluminium,niobium,vanadium,gauge,width,weight,location,defect,linear feat,defect_code
index,,,,,,,,,,,,,,,,
684778-06AA,30.0,0.0038,0.332,0.015,0.0006,0.300,0.0069,0.005,0.054,0.0411,45.347,21350.0,"PRECISION STRIP (MIDDLETOWN) Middletown, OH",0616 COATING SCRATCHES,3512.0,0616
681110-02A,30.0,0.0038,0.332,0.015,0.0006,0.300,0.0069,0.005,0.054,0.0572,48.358,38470.0,"AK STEEL CORPORATION Middletown, OH",0606 COATING DENTS,4232.0,0606
686359-01,30.0,0.0052,0.431,0.016,0.0006,0.317,0.0074,0.001,0.058,0.0370,48.000,36870.0,"PRECISION STRIP (MIDDLETOWN) Middletown, OH",0636 COATING DROSS,5774.0,0636
684481-04BA,30.0,0.0096,0.340,0.023,0.0013,0.390,0.0080,0.017,0.066,0.0470,48.858,9140.0,"AK STEEL CORPORATION Middletown, OH",0933 LITE COIL-QUALITY,1213.0,0933
684592-01AA,30.0,0.0100,0.310,0.023,0.0015,0.370,0.0080,0.024,0.065,0.0566,44.582,11850.0,"AK STEEL CORPORATION Middletown, OH",2939 DISTRESSED INV PRV DIV,1426.0,2939


In [4]:
acer_vars = ['gauge', 'width', 'weight', 'carbon', 'manganese', 'sulfur', 'phosphorus', 'silicon', 'niobium',
'vanadium', 'aluminium'] # all variables included in acermital & ak together

# Creating pipeline for ak_steel webiste

Creating class to train a perceptron model based off th data

In [73]:
class ak_steel_perceptron_model:
    
    
    
    def __init__(self, data = ak_model_total, variables = acer_vars):
        
        self.ak_clean_data = data.dropna()
        
        
        self.train_data, self.test_data = train_test_split(self.ak_clean_data)
        self.vars = variables # defining variables

    def create_mlp(self):
        # pulling values to be scaled 
       
        X_train = self.train_data.loc[:, self.vars].values
        
        y_train = self.train_data.loc[:, "price"].values

        X_test = self.test_data.loc[:, self.vars].values


        y_test = self.test_data.loc[:, "price"].values

    
    
        # standardizing input data for mlp regression 
    

        x_scaler = StandardScaler() 
        
        x_scaler.fit(X_train)  

        X_train_scaled = x_scaler.transform(X_train)  
        
        X_test_scaled = x_scaler.transform(X_test)  
        
        
        
        # standardizing output data for mlp regression 

        y_train = y_train.reshape(-1,1)

        y_scaler = StandardScaler()
        

        
        y_scaler.fit(y_train)  

        y_train_scaled = y_scaler.transform(y_train)  

        # apply same transformation to test data

        y_test = y_test.reshape(-1,1)


        y_test_scaled = y_scaler.transform(y_test)

        
        
        
        
        ak_regressor = MLPRegressor(max_iter=500)


        
    
        self.y_scaler = y_scaler
        
        self.mlp_fitted = ak_regressor.fit(X_train_scaled, np.ravel(y_train_scaled))
        
        self.rscore = self.mlp_fitted.score(X_test_scaled, np.ravel(y_test_scaled))


    
        
        
        return self

        
        
#     def get_mlp_prediction(self, index): # return prediction based off of index_code/identifier
        
#         prediction = self.y_scaler.inverse_transform(self.mlp_fitted.predict(self.ak_clean_data.loc[index].reshape(1,-1)))

        
        
#         return prediction
            
            
        
        
trained_model = ak_steel_perceptron_model()

trained_model.create_mlp()


trained_model.rscore

# kk.get_mlp_prediction(40)




0.7903628722531634

# Creating pipline to download and open data from acermital targetsteel excel file

# create a function to download all 30 excel files, and concat them into a single excel file to be downloaded

In [74]:
acermital_data_pretty = pd.read_excel('test_acermital_df.xlsx', skiprows=6, index_col='Piece ID / Ref#')



acermital_data_model = acermital_data_pretty.drop(columns = ['Minor Product', 'Business Unit',
       'Category', 'Location', 'Grade', 'Comment', 'Defect', 'Trans#'])

acermital_data_model.rename(columns = {'Gauge':'gauge', 'Width':'width', 'Weight':'weight'}, inplace = True)


In [75]:
acermital_data_model.head()

,gauge,width,weight,Chemistry
Piece ID / Ref#,,,,
960-51209 / 853498,0.0390 x 0.0390,36.0561,11662,C: .0291 MN: .2091 S: .0099 P: .0107 SI: .0178...
854-89507 / 955005,0.0552 x 0.0552,48.5000,15800,C: .0290 MN: .1919 S: .0071 P: .0068 SI: .0091...
854-90173 / 960985,0.0579 x 0.0579,35.9375,16040,C: .0247 MN: .2346 S: .0106 P: .0115 SI: .0084...
854-90172 / 960985,0.0579 x 0.0579,35.9375,17740,C: .0247 MN: .2346 S: .0106 P: .0115 SI: .0084...
9482391 / 843117,0.0753 x 0.0753,34.3238,29480,C: .0883 MN:1.0170 S: .0039 P: .0110 SI: .3130...


In [76]:
# fixing gauge issue

def gauge_fix():
    gauge_data = acermital_data_model['gauge'].values
    
    pretty_gauges = []
    for i in gauge_data:
        pretty_gauges.append(float(i.split()[0]))
    
    return pretty_gauges


acermital_data_model['gauge'] = gauge_fix()

In [77]:

def chem_fix():
    chem_list = acermital_data_model['Chemistry'].values.tolist()


    chem_vars = acer_vars[3:]

    chem_dict = {k:None for k in chem_vars }

    chem_clean = []
    for i in chem_list:
        chem_clean.append(i.replace(":",": ").split())

    chem_array = np.array(chem_clean)

    chem_dict['carbon'] = chem_array[:, 1].astype(float)

    chem_dict['manganese'] = chem_array[:, 3].astype(float)
    chem_dict['sulfur'] = chem_array[:, 5].astype(float)
    chem_dict['phosphorus'] = chem_array[:, 7].astype(float)
    chem_dict['silicon'] = chem_array[:, 9].astype(float)
    chem_dict['niobium'] = chem_array[:, 11].astype(float)
    chem_dict['vanadium'] = chem_array[:, 13].astype(float)
    chem_dict['aluminium'] = chem_array[:, 15].astype(float)

    return chem_dict

In [78]:
acermital_data_model['carbon'] = chem_fix()['carbon'] 
acermital_data_model['sulfur'] = chem_fix()['sulfur']
acermital_data_model['manganese'] = chem_fix()['sulfur']

acermital_data_model['phosphorus'] = chem_fix()['phosphorus']
acermital_data_model['silicon'] = chem_fix()['silicon']
acermital_data_model['niobium'] = chem_fix()['niobium']
acermital_data_model['vanadium'] = chem_fix()['vanadium']
acermital_data_model['aluminium'] = chem_fix()['aluminium']


In [79]:


acermital_data_model

,gauge,width,weight,Chemistry,carbon,sulfur,manganese,phosphorus,silicon,niobium,vanadium,aluminium
Piece ID / Ref#,,,,,,,,,,,,
960-51209 / 853498,0.0390,36.0561,11662,C: .0291 MN: .2091 S: .0099 P: .0107 SI: .0178...,0.0291,0.0099,0.0099,0.0107,0.0178,0.0004,0.0002,0.0458
854-89507 / 955005,0.0552,48.5000,15800,C: .0290 MN: .1919 S: .0071 P: .0068 SI: .0091...,0.0290,0.0071,0.0071,0.0068,0.0091,0.0005,0.0002,0.0434
854-90173 / 960985,0.0579,35.9375,16040,C: .0247 MN: .2346 S: .0106 P: .0115 SI: .0084...,0.0247,0.0106,0.0106,0.0115,0.0084,0.0005,0.0002,0.0445
854-90172 / 960985,0.0579,35.9375,17740,C: .0247 MN: .2346 S: .0106 P: .0115 SI: .0084...,0.0247,0.0106,0.0106,0.0115,0.0084,0.0005,0.0002,0.0445
9482391 / 843117,0.0753,34.3238,29480,C: .0883 MN:1.0170 S: .0039 P: .0110 SI: .3130...,0.0883,0.0039,0.0039,0.0110,0.3130,0.0020,0.0020,0.0480
9482388 / 844146,0.0754,34.2682,30280,C: .0904 MN:1.0150 S: .0046 P: .0090 SI: .3050...,0.0904,0.0046,0.0046,0.0090,0.3050,0.0010,0.0020,0.0490
9486812 / 844150,0.0756,34.3932,25600,C: .0865 MN:1.0200 S: .0056 P: .0090 SI: .3050...,0.0865,0.0056,0.0056,0.0090,0.3050,0.0010,0.0020,0.0470
9489715 / 843116,0.0761,34.2682,27980,C: .0842 MN: .9680 S: .0060 P: .0090 SI: .2590...,0.0842,0.0060,0.0060,0.0090,0.2590,0.0010,0.0010,0.0370
9501023 / 844148,0.0764,34.2682,30140,C: .0854 MN:1.0260 S: .0059 P: .0130 SI: .2980...,0.0854,0.0059,0.0059,0.0130,0.2980,0.0020,0.0020,0.0460


In [80]:
acermital_data_model_final = acermital_data_model.drop(columns = ['Chemistry'])

acer_cols = acermital_data_model_final.columns.tolist()

In [83]:
trained_model

In [69]:
trained_model

NameError: name 'y_scaler' is not defined

In [70]:
trained_model.y_scaler.inverse_transform(trained_model.mlp_fitted.predict(self.ak_clean_data.loc[index].reshape(1,-1)))


NameError: name 'self' is not defined

# Creating code for classifer

In [ ]:

X_train_class = train.loc[:, ['carbon', 'manganese', 'phosphorus',
              'sulfur', 'silicon', 'aluminium', 'niobium',
              'vanadium', 'gauge', 'width',
              'weight', 'linear feat', 'price']].values

y_train_class = train.loc[:, "defect_code"].values



X_test_class = test.loc[:, ['carbon', 'manganese', 'phosphorus',
              'sulfur', 'silicon', 'aluminium', 'niobium',
              'vanadium', 'gauge', 'width',
              'weight', 'linear feat', 'price']].values



y_test_class = test.loc[:, "defect_code"].values






# standardizing input data for mlp classification 

x_scaler_class = StandardScaler()  
x_scaler_class.fit(X_train_class)  

X_train_class = x_scaler_class.transform(X_train_class)  

# apply same transformation to test data
X_test_class = x_scaler_class.transform(X_test_class)  





# labeling output data for mlp regression 

y_train_class = y_train_class.reshape(-1,1)

encoder = OneHotEncoder(handle_unknown="ignore")

encoder.fit(y_train_class)

y_train_class = encoder.transform(y_train_class)





# apply same transformation to test data

y_test_class = y_test_class.reshape(-1,1)

y_test_class = encoder.transform(y_test_class)


In [ ]:
# labeling y data

# Create the encoder.

    # Assume for simplicity all features are categorical.

# Apply the encoder.


In [ ]:
ak_classifer = MLPClassifier()

ak_classifer.fit(X_train_class, y_train_class)

ak_classifer.score(X_test_class, y_test_class)